In [1]:
import pandas as pd
import wbgapi as wb
import plotly.express as px
from utils.config import Rabet_bg_color,Rabet_color_palette
from prophet import Prophet

theme_color = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]

/opt/anaconda3/envs/mydev/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def getCo2Emission():
    df_income = wb.data.DataFrame('EN.GHG.CH4.WA.MT.CE.AR5',
                                    economy=wb.region.members('AFR'),labels=True)
    df_income = df_income.reset_index(drop=True).set_index('Country')
    df_income = df_income.iloc[1:,-20:-3].reset_index()
    df_income.loc[60] = df_income.sum(numeric_only=True)
    df_income['Country'] = df_income['Country'].fillna('Africa')
    df_income['Diff_Gain'] = df_income['YR2004']-df_income['YR2020']
    df_income.sort_values(by='YR2020', inplace=True,ascending=False)
    df_income = df_income.round(2)
    return df_income.reset_index(drop=True)
df_getCo2Emission=getCo2Emission()
df_getCo2Emission.head()

/opt/anaconda3/envs/mydev/lib/python3.10/site-packages/wbgapi/data.py:327: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummy = pd.Series()    # empty series - never assigned actual values


,Country,YR2004,YR2005,YR2006,YR2007,YR2008,YR2009,YR2010,YR2011,YR2012,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,Diff_Gain
0,Africa,201.76,207.64,210.25,215.78,222.52,228.06,235.66,242.86,252.55,260.06,267.46,275.10,282.39,290.49,298.46,306.61,314.31,-112.55
1,Nigeria,25.76,26.67,27.68,28.77,29.92,31.16,32.46,33.75,35.10,36.52,38.01,39.40,40.61,41.95,43.27,44.56,45.89,-20.13
2,South Africa,20.96,21.46,22.23,22.73,23.35,23.01,23.66,24.11,25.90,26.76,26.78,26.88,27.01,27.43,27.65,27.98,28.09,-7.13
3,Morocco,15.25,15.63,16.15,16.72,17.00,17.49,18.01,18.58,19.10,19.68,19.94,20.56,20.98,21.52,22.02,22.51,22.99,-7.74
4,"Egypt, Arab Rep.",16.36,16.72,17.21,16.91,17.38,17.66,18.37,18.95,19.77,19.69,20.07,20.33,20.71,21.45,21.87,22.43,22.94,-6.58


In [129]:
# wb.source.info() # especially useful for seeing what databases are available
# wb.series.info(db=77)
# LIST REGIONS
# wb.region.info()
# wb.economy.info('CAN')
# wb.economy.info(['USA', 'MEX', 'ARG','KEN'])
wb.series.info(q='tax')
# wb.economy.info(q='korea')
# wb.series.metadata.get('EN.ATM.CO2E.PC')

id,value
GC.TAX.EXPT.CN,Taxes on exports (current LCU)
GC.TAX.EXPT.ZS,Taxes on exports (% of tax revenue)
GC.TAX.GSRV.CN,Taxes on goods and services (current LCU)
GC.TAX.GSRV.RV.ZS,Taxes on goods and services (% of revenue)
GC.TAX.GSRV.VA.ZS,Taxes on goods and services (% value added of industry and services)
GC.TAX.INTT.CN,Taxes on international trade (current LCU)
GC.TAX.INTT.RV.ZS,Taxes on international trade (% of revenue)
GC.TAX.OTHR.CN,Other taxes (current LCU)
GC.TAX.OTHR.RV.ZS,Other taxes (% of revenue)
GC.TAX.TOTL.CN,Tax revenue (current LCU)


In [124]:
df_cereal = wb.data.DataFrame('SI.RMT.COST.IB.ZS',economy=wb.region.members('AFR'), labels=True)
df_cereal.head(60)

/opt/anaconda3/envs/mydev/lib/python3.10/site-packages/wbgapi/data.py:327: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummy = pd.Series()    # empty series - never assigned actual values


,Country,YR1960,YR1961,YR1962,YR1963,YR1964,YR1965,YR1966,YR1967,YR1968,...,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022,YR2023
economy,,,,,,,,,,,,,,,,,,,,,
GNB,Guinea-Bissau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MWI,Malawi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,14.158610,12.183120,11.193140,15.725430,9.395780,7.590000,-10.493334,-49.693334
BDI,Burundi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SEN,Senegal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.473810,4.405425,3.454165,4.223810,3.547615,3.614165,1.328334,0.720000
GMB,"Gambia, The",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,13.642490,6.101540,6.098870,6.358860,1.699850,7.260000,5.533333,6.760000
NER,Niger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EGY,"Egypt, Arab Rep.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.275791,6.215062,4.303596,4.911591,4.705607,4.774050,3.930278,3.303095
BWA,Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19.202220,12.092670,10.556740,12.618240,10.017260,10.060000,6.893333,8.412857
MDG,Madagascar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.420420,9.872870,NaN,12.800850,14.485830,16.595000,8.420000,2.925000


In [98]:
df_cereal_new = pd.melt(df_cereal, id_vars=['Country'],var_name='ds', value_name='y')

df_cereal_new['ds'] = df_cereal_new['ds'].apply(lambda x: x[-4:])

df_cereal_new.head(100)

,Country,ds,y
0,Guinea-Bissau,1960,25569.0
1,Malawi,1960,50199.0
2,Burundi,1960,45564.0
3,Senegal,1960,359097.0
4,"Gambia, The",1960,37068.0
...,...,...,...
95,Eritrea,1961,130858.0
96,Kenya,1961,315553.0
97,Benin,1961,77928.0
98,Tanzania,1961,174346.0


In [99]:
countries = df_cereal_new['Country'].unique()
loaded_countries = []
# for i in countries:
#     print(i)

In [100]:
for i in range(len(countries)):
    try:
        country_df = df_cereal_new[df_cereal_new['Country'] == countries[i]]
        my_model = Prophet(interval_width=0.95)
        my_model.fit(country_df)
        future_dates = my_model.make_future_dataframe(periods=10, freq='Y',include_history=False)
        forecast = my_model.predict(future_dates)
        final_forecast = forecast[['ds', 'yhat']].copy()
        final_forecast['Country'] = countries[i]
        loaded_countries.append(final_forecast)
    except Exception as e:
        print(f"An error occured: {e}")

18:01:51 - cmdstanpy - INFO - Chain [1] start processing
18:01:51 - cmdstanpy - INFO - Chain [1] done processing
18:01:51 - cmdstanpy - INFO - Chain [1] start processing
18:01:52 - cmdstanpy - INFO - Chain [1] done processing
18:01:52 - cmdstanpy - INFO - Chain [1] start processing
18:01:52 - cmdstanpy - INFO - Chain [1] done processing
18:01:52 - cmdstanpy - INFO - Chain [1] start processing
18:01:52 - cmdstanpy - INFO - Chain [1] done processing
18:01:52 - cmdstanpy - INFO - Chain [1] start processing
18:01:53 - cmdstanpy - INFO - Chain [1] done processing
18:01:53 - cmdstanpy - INFO - Chain [1] start processing
18:01:53 - cmdstanpy - INFO - Chain [1] done processing
18:01:53 - cmdstanpy - INFO - Chain [1] start processing
18:01:54 - cmdstanpy - INFO - Chain [1] done processing
18:01:54 - cmdstanpy - INFO - Chain [1] start processing


An error occured: Dataframe has less than 2 non-NaN rows.


18:01:54 - cmdstanpy - INFO - Chain [1] done processing
18:01:54 - cmdstanpy - INFO - Chain [1] start processing
18:01:55 - cmdstanpy - INFO - Chain [1] done processing
18:01:55 - cmdstanpy - INFO - Chain [1] start processing


An error occured: Dataframe has less than 2 non-NaN rows.
An error occured: Dataframe has less than 2 non-NaN rows.


18:01:55 - cmdstanpy - INFO - Chain [1] done processing
18:01:55 - cmdstanpy - INFO - Chain [1] start processing
18:01:56 - cmdstanpy - INFO - Chain [1] done processing
18:01:56 - cmdstanpy - INFO - Chain [1] start processing
18:01:56 - cmdstanpy - INFO - Chain [1] done processing
18:01:56 - cmdstanpy - INFO - Chain [1] start processing
18:01:57 - cmdstanpy - INFO - Chain [1] done processing
18:01:57 - cmdstanpy - INFO - Chain [1] start processing
18:01:57 - cmdstanpy - INFO - Chain [1] done processing
18:01:57 - cmdstanpy - INFO - Chain [1] start processing
18:01:57 - cmdstanpy - INFO - Chain [1] done processing
18:01:57 - cmdstanpy - INFO - Chain [1] start processing


...

An error occured: Dataframe has less than 2 non-NaN rows.


18:02:05 - cmdstanpy - INFO - Chain [1] done processing
18:02:05 - cmdstanpy - INFO - Chain [1] start processing
18:02:05 - cmdstanpy - INFO - Chain [1] done processing
18:02:05 - cmdstanpy - INFO - Chain [1] start processing
18:02:06 - cmdstanpy - INFO - Chain [1] done processing
18:02:06 - cmdstanpy - INFO - Chain [1] start processing
18:02:06 - cmdstanpy - INFO - Chain [1] done processing
18:02:06 - cmdstanpy - INFO - Chain [1] start processing
18:02:06 - cmdstanpy - INFO - Chain [1] done processing
18:02:06 - cmdstanpy - INFO - Chain [1] start processing
18:02:07 - cmdstanpy - INFO - Chain [1] done processing
18:02:07 - cmdstanpy - INFO - Chain [1] start processing
18:02:07 - cmdstanpy - INFO - Chain [1] done processing
18:02:07 - cmdstanpy - INFO - Chain [1] start processing
18:02:08 - cmdstanpy - INFO - Chain [1] done processing
18:02:08 - cmdstanpy - INFO - Chain [1] start processing
18:02:08 - cmdstanpy - INFO - Chain [1] done processing
18:02:08 - cmdstanpy - INFO - Chain [1] 

An error occured: Dataframe has less than 2 non-NaN rows.


18:02:10 - cmdstanpy - INFO - Chain [1] done processing
18:02:10 - cmdstanpy - INFO - Chain [1] start processing
18:02:10 - cmdstanpy - INFO - Chain [1] done processing
18:02:10 - cmdstanpy - INFO - Chain [1] start processing
18:02:11 - cmdstanpy - INFO - Chain [1] done processing
18:02:11 - cmdstanpy - INFO - Chain [1] start processing
18:02:11 - cmdstanpy - INFO - Chain [1] done processing


An error occured: Dataframe has less than 2 non-NaN rows.


In [101]:
for i in loaded_countries:
    print(i.head())

          ds           yhat        Country
0 2023-12-31  672475.277961  Guinea-Bissau
1 2024-12-31  697137.597812  Guinea-Bissau
2 2025-12-31  715577.717972  Guinea-Bissau
3 2026-12-31  733626.578436  Guinea-Bissau
4 2027-12-31  751294.845149  Guinea-Bissau
          ds          yhat Country
0 2023-12-31  1.270065e+06  Malawi
1 2024-12-31  1.328149e+06  Malawi
2 2025-12-31  1.365250e+06  Malawi
3 2026-12-31  1.401018e+06  Malawi
4 2027-12-31  1.435464e+06  Malawi
          ds          yhat  Country
0 2023-12-31  1.183598e+06  Burundi
1 2024-12-31  1.255229e+06  Burundi
2 2025-12-31  1.298134e+06  Burundi
3 2026-12-31  1.339185e+06  Burundi
4 2027-12-31  1.378394e+06  Burundi
          ds          yhat  Country
0 2023-12-31  3.421636e+06  Senegal
1 2024-12-31  3.521268e+06  Senegal
2 2025-12-31  3.590006e+06  Senegal
3 2026-12-31  3.656608e+06  Senegal
4 2027-12-31  3.721045e+06  Senegal
          ds           yhat      Country
0 2023-12-31  474875.459100  Gambia, The
1 2024-12-31  4837

In [9]:
def getCo2Emission():
    # DEBUG AND RETURN 'E   N.ATM.CO2E.PC'
    df_income = wb.data.DataFrame('EN.GHG.CH4.WA.MT.CE.AR5',
                                    economy=wb.region.members('AFR'),labels=True)
    df_income = df_income.reset_index(drop=True).set_index('Country')
    df_income = df_income.iloc[1:,-20:-3].reset_index()
    df_income.loc[60] = df_income.sum(numeric_only=True)
    df_income['Country'] = df_income['Country'].fillna('Africa')
    df_income['Diff_Gain'] = df_income['YR2005']-df_income['YR2021']
    df_income.sort_values(by='YR2020', inplace=True,ascending=False)
    df_income = df_income.round(2)
    return df_income.reset_index(drop=True)
df_getCo2Emission=getCo2Emission()

/opt/anaconda3/envs/mydev/lib/python3.10/site-packages/wbgapi/data.py:327: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummy = pd.Series()    # empty series - never assigned actual values


In [10]:
df_getCo2Emission.head()

,Country,YR2005,YR2006,YR2007,YR2008,YR2009,YR2010,YR2011,YR2012,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,Diff_Gain
0,Africa,124.65,125.21,128.46,132.42,136.20,141.27,145.93,152.94,157.68,162.25,167.12,172.14,177.70,183.13,188.01,192.91,200.97,-76.33
1,Nigeria,16.10,16.79,17.54,18.23,19.22,20.15,21.05,22.01,23.02,24.08,25.06,25.83,26.73,27.59,28.43,29.30,30.17,-14.07
2,South Africa,15.85,16.50,16.91,17.44,17.17,17.70,18.12,19.82,20.59,20.57,20.58,20.65,21.01,21.18,21.45,21.54,21.87,-6.02
3,Morocco,12.70,13.19,13.69,13.95,14.40,14.88,15.41,15.88,16.43,16.70,17.27,17.67,18.17,18.64,19.09,19.53,19.98,-7.28
4,Algeria,10.60,10.92,11.31,11.67,12.06,12.52,12.97,13.46,13.90,14.58,15.19,15.54,15.85,16.51,16.92,17.21,20.12,-9.52


In [6]:
def getPctForestArea():
    df_income = wb.data.DataFrame('AG.LND.FRST.ZS',
                                  economy=wb.region.members('AFR'),labels=True)
    df_income = df_income.reset_index(drop=True).set_index('Country')
    df_income = df_income.iloc[1:,-20:-2].reset_index()
    df_income['Pct_Lost'] = df_income['YR2006']-df_income['YR2023']
    df_income.loc[60] = df_income.mean(numeric_only=True)
    df_income['Country'] = df_income['Country'].fillna('Africa')
    df_income.sort_values(by='YR2023', inplace=True,ascending=False)
    return df_income.reset_index(drop=True)

df_PctForestArea = getPctForestArea()
df_PctForestArea.head()

/opt/anaconda3/envs/mydev/lib/python3.10/site-packages/wbgapi/data.py:327: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummy = pd.Series()    # empty series - never assigned actual values


,Country,YR2006,YR2007,YR2008,YR2009,YR2010,YR2011,YR2012,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022,YR2023,Pct_Lost
0,Gabon,91.860310,91.840676,91.821042,91.801409,91.781775,91.735662,91.689549,91.643435,91.597322,91.551209,91.505103,91.458998,91.412892,91.366787,91.320681,91.274576,91.228471,91.182365,0.677945
1,Equatorial Guinea,91.459465,91.161533,90.863601,90.565668,90.267736,89.969768,89.671800,89.373832,89.075865,88.777897,88.479857,88.181818,87.883779,87.585740,87.287701,86.989661,86.691622,86.393583,5.065882
2,Liberia,83.482973,83.168812,82.854651,82.540490,82.226329,81.912168,81.598007,81.283846,80.969684,80.655523,80.341362,80.027201,79.713040,79.398879,79.084718,78.770556,78.456395,78.142234,5.340739
3,Seychelles,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,73.260870,0.000000
4,Guinea-Bissau,74.616287,74.315967,74.015647,73.715327,73.415007,73.114723,72.814438,72.514154,72.213869,71.913585,71.613442,71.313300,71.013158,70.713016,70.412873,70.112731,69.812589,69.512447,5.103841
